In [ ]:
# 步驟
# 對 XY 
    # 做高斯模糊
    # 把最大與最小1%的數值都推到極值，中間數值等比例擴張，再依照前十層的平均亮度去調整亮度
# 對 XZ 
    # Three Step Pyramid 提升邊界特徵，降躁
    # 也有把最小的數值(黑色的部分)都推到極值
# 對 XZ 做正規化 (0~1)
    # 再推一次極值 1 %


In [ ]:
import tifffile
from csbdeep.utils import normalize

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
input_name = r"C:/our_data/D_20220811/20211124_151257C.bin"
print(input_name)

save_process = False
print_process = True
log_value = False

output_name = input_name.replace(".bin","_norm3D_py.tif")
# output_name = r"D:/git/our_data/20210308_095750C_normalize_python.tif"
print(output_name)

In [ ]:
# 設定 / 備註
cut_mid = False #True
if cut_mid :
    # region 直接在 write_output 裡面的 funtion 設定
    output_name = output_name.replace(".tif","_mid.tif")

human_see = False 
if human_see :
    output_name_human = input_name.replace(".bin","_human.tif")

print("output_name :",output_name)


In [ ]:
file_name_count = 1
def write_output(pic, name = None):
    global file_name_count
    global cut_mid
    if "int" in str(pic.dtype) :
        print("in write_output do normalize")
        from csbdeep.utils import normalize
        pic = normalize(pic, 1, 99.8, axis= (0,1,2) )
    global output_name
    if name == None :
        save_name = output_name.replace(".tif","_"+str(file_name_count)+".tif")
        file_name_count += 1
    else :
        save_name = output_name.replace(".tif","_"+str(name)+".tif")
    print("save name :", save_name)
    if cut_mid :
        tifffile.imsave(save_name, pic[:,:500,:500])
    else :
        tifffile.imsave(save_name, pic)

In [ ]:
if "bin" in input_name :
    f = open(input_name, "rb")   # open the bin file
    if "processed" in input_name :
        data_pic = np.fromfile(f, dtype=np.float32, count=-1, offset=0)
    else :
        data_pic = np.fromfile(f, dtype=np.int32, count=-1, offset=0)
    
    print("do reshape")
    data_pic = data_pic.reshape((-1,1024,1024))

In [ ]:
# 轉float
data_pic = data_pic.astype("float32")

In [ ]:
if log_value :
    # 確認
    print(data_pic.shape)
    print(data_pic.dtype)
    print(data_pic[100])

if print_process :
    plt.imshow(data_pic[100], cmap='gray')

In [ ]:
# tag 0 XY 前處理 (高斯模糊)
# 對 XY 做高斯模糊
from scipy.ndimage import gaussian_filter
sigma = 1
Image_Stack_Blur = np.array([
    gaussian_filter(data_pic[i], sigma=sigma, mode = 'nearest', truncate = np.ceil(2*sigma)/sigma )  # imgaussfilt
        for i in range(data_pic.shape[0])])

# 3D 一起做效果應該會比較好 ??
# Image_Stack_Blur = gaussian_filter(data_pic, sigma=sigma, mode = 'nearest',truncate=np.ceil(2*sigma)/sigma)

In [ ]:
if log_value :
    # 確認
    print(Image_Stack_Blur.shape)
    print(Image_Stack_Blur.dtype)
    print(Image_Stack_Blur[100])

if print_process :
    plt.imshow(Image_Stack_Blur[100], cmap='gray')
    
if save_process :
    write_output(Image_Stack_Blur, name = "gaussian_filter")

In [ ]:
# print(Image_Stack_Blur[:,:,300])
# plt.imshow(Image_Stack_Blur[:,:,300], cmap='gray')

In [ ]:
# tag 1 XY 前處理 (參數)
Adj1_A=1
Adj2_A=1
Buffer_Length=10
Cmin_Buffer_Array=np.zeros(Buffer_Length)
Cmax_Buffer_Array=np.zeros(Buffer_Length)

In [ ]:
Image_Stack_Blur_AHist_Corr = np.zeros( shape = data_pic.shape , dtype=data_pic.dtype ) 
# tag 2 XY 前處理
# 把前後1%的數值都推到極值，中間數值等比例擴張，再依照前十層的平均亮度去調整亮度
# 詳細作法
    # 從中間的範圍找出 前後1%的值(Cmin Cmax)
    # 會先把數值調整到 接近0~1之間(因為是Cmin所以會超出) 
        # ((Current_XY_Image - Cmin_Now)/(Cmax_Now - Cmin_Now))
    # 超出的就強制設定成 0 或 1
    # 再計算前10張的亮度的數值 反向還原 (Image_Temp*(Cmax_Buffered-Cmin_Buffered)+Cmin_Buffered)

# Cmin_Buffered 應該要用原圖的數值 還是 調整後的數值 ?? (不過影響應該不大)

for p in range(data_pic.shape[0]):
    Current_XY_Image = Image_Stack_Blur[p]
    ROI_Image = Current_XY_Image[49:-50,49:-50]  # index 問題
    # print(ROI_Image.shape)
    # 變成一維才能sort
    ROI_Image = ROI_Image.flatten()
    Sort_ascend = np.sort(ROI_Image)
    # 如果要優化應該要用 heap ?

    get_place = round(len(Sort_ascend)*Adj1_A/100.0)-1  # index 問題

    Cmin_Now = Sort_ascend[get_place]
    Cmax_Now = Sort_ascend[-get_place + 1]

    Cmin_Buffer_Array[p%Buffer_Length] = Cmin_Now
    Cmax_Buffer_Array[p%Buffer_Length] = Cmax_Now

    Cmin_Buffered = np.mean(Cmin_Buffer_Array[np.nonzero(Cmin_Buffer_Array)])
    Cmax_Buffered = np.mean(Cmax_Buffer_Array[np.nonzero(Cmax_Buffer_Array)])

    Image_Temp=(Current_XY_Image - Cmin_Now)/(Cmax_Now - Cmin_Now) 
    # Image_Temp 在接近 0~1 之間(會小於0 大於1)
    Image_Adjed = Image_Temp*(Cmax_Buffered-Cmin_Buffered)+Cmin_Buffered
    # 前10張的亮度的平均數值 使用上面那一行相反的運算方式還原回去
    Image_Adjed = np.clip(Image_Adjed, a_min = 0, a_max=None)

    Image_Stack_Blur_AHist_Corr[p] = Image_Adjed


In [ ]:
if log_value :
    # 確認
    print(Image_Stack_Blur_AHist_Corr.shape)
    print(Image_Stack_Blur_AHist_Corr.dtype)
    print(Image_Stack_Blur_AHist_Corr[100])

if print_process :
    plt.imshow(Image_Stack_Blur_AHist_Corr[100], cmap='gray')

if save_process :
    write_output(Image_Stack_Blur_AHist_Corr, "extreme")

In [ ]:
# tag 4.0 XZ 前處理(參數)
 
# Median_1=1
# Median_2=1
Blur_1=5
Blur_2=20
Exp_1=1
Exp_2=1.5
Exp_3=0.45

In [ ]:
# 這裡開始對 XZ plane 做處理
from math import log

Processed_B_Stack = np.zeros( shape = data_pic.shape , dtype=data_pic.dtype ) 
# tag 3 XZ 前處理
for p in range(Image_Stack_Blur_AHist_Corr.shape[2]):
    Image_XZ_Original = Image_Stack_Blur_AHist_Corr[:,p,:]
    # Image_Log = Image_XZ_Original
    Image_Log = np.log(Image_XZ_Original)
    Image_Log = np.where( Image_Log == -np.inf, np.nan, Image_Log ) # 因為 log(0) 會是負無限大 保險起見把這些值去除
    Image_Log_flat = Image_Log.flatten() 
    Image_Log_Sort_1D = np.sort(Image_Log_flat)
    Image_Log_Sort_1D_front = Image_Log_Sort_1D[0:round(Image_Log_Sort_1D.shape[0]/10.0)-1]
    # Min_by_Sort : 前 10 % 數值的平均 # 可以想成 Cmin
    Min_by_Sort = np.nanmean(Image_Log_Sort_1D_front)
    Image_Log = Image_Log - Min_by_Sort
    Image_Log = np.where( Image_Log == np.nan, 0, Image_Log)
    Image_Log = np.clip(Image_Log, a_min = 0.0000001, a_max=None)

    # tag 4 XZ  log 完後 模糊處理
    Image_Blur_1 = gaussian_filter(Image_Log, sigma = Blur_1, mode = 'nearest', truncate = np.ceil(2*Blur_1)/Blur_1)
    Image_Blur_2 = gaussian_filter(Image_Log, sigma = Blur_2, mode = 'nearest', truncate = np.ceil(2*Blur_2)/Blur_2)

    from scipy.signal import medfilt2d  # 中值濾波 
    
    # Three Step Pyramid
    # 1st: high resolution 
    # 2nd: mid resolution that we care
    # 3rd: low resolution 
    # 將深處低解析度之訊號強化(band 1)
    # 不過度影響淺層高解析度影像之對比(band 2)
    # 且不將高頻speckle noise放大(band 3)
    # http://persci.mit.edu/pub_pdfs/RCA84.pdf (37頁右邊)

    Image_1st_Backup = Image_Log-Image_Blur_1 # 邊緣的資訊
    # Image_1st_Backup = medfilt2d(Image_1st_Backup,(Median_1, Median_1))
    Image_1st = Image_1st_Backup**Exp_1
    Image_1st = (Image_1st/np.max(Image_1st))*np.max(Image_1st_Backup)

    # print("np.min(Image_Blur_2)", np.min(Image_Blur_2))
    Image_2nd_Backup = Image_Blur_1 / Image_Blur_2  
    # Image_2nd_Backup = medfilt2d(Image_2nd_Backup,(Median_2, Median_2))
    Image_2nd = Image_2nd_Backup**Exp_2
    Image_2nd = (Image_2nd/np.max(Image_2nd))*np.max(Image_2nd_Backup)

    Image_3rd_Backup = Image_Blur_2
    Image_3rd = Image_3rd_Backup**Exp_3
    Image_3rd = (Image_3rd / np.max(Image_3rd)) * np.max(Image_3rd_Backup)

    Image_Rec = Image_3rd * Image_2nd + Image_1st    # *Image_1st

    Processed_B_Stack[:,p,:]=Image_Rec

In [ ]:
if log_value :
    print("Image_1st")
    print(Image_1st)
    print("Image_2nd")
    print(Image_2nd)
    print("Image_3rd")
    print(Image_3rd)

In [ ]:
if log_value :
    # 確認
    print(Processed_B_Stack.shape)
    print(Processed_B_Stack.dtype)
    print(Processed_B_Stack[100])

if print_process :
    plt.imshow(Processed_B_Stack[100], cmap='gray')

if save_process :
    write_output(Processed_B_Stack,"Processed_B_Stack")

In [ ]:
# tag 5 最後處理，輸出

# 把數值調整到 0~1 之間 (超過的數值會調整)
Adj1=1 
Adj2=1 #0.3

In [ ]:
Processed_B_Stack = Image_Stack_Blur_AHist_Corr
Histogram_Corr_Image_Stack = Processed_B_Stack.copy()
# 還是對 XZ, 做正規化 
for p in range(Processed_B_Stack.shape[1]):
    Current_Image = Processed_B_Stack[:,p,:]
    ROI_Image=Current_Image[29:,99:925]
    ROI_Image = ROI_Image.flatten()
    Sort_ascend = np.sort(ROI_Image)
    get_place = round(len(Sort_ascend)*Adj1/100.0)-1 # Cmin 的位置 是 Adj1 %
    Cmin = Sort_ascend[get_place]
    Cmax = Sort_ascend[ -get_place + 1 ]*1.15 
    # 這裡 Cmax 有點尷尬 如果不*1.15 Image_Norm會有值到 1.5 左右
    # 因為Cmin跟Cmax都很大而且很接近 所以如果Cmax*1.15分母會突然變得很大

    Image_Norm=(Current_Image-Cmin) / (Cmax-Cmin)
    if log_value and p % 10 == 0:
        print(p, Cmax, Cmin)
        print(p, np.max(Image_Norm))
    Image_Norm = np.clip(Image_Norm, a_min = 0, a_max=1)
    Histogram_Corr_Image_Stack[:,p,:] = Image_Norm

In [ ]:
# 看一下最終結果
plt.imshow(Histogram_Corr_Image_Stack[100], cmap='gray')

In [ ]:
write_output(Histogram_Corr_Image_Stack)

In [ ]:
if human_see :
    human_result = np.zeros( shape = data_pic.shape , dtype=data_pic.dtype )
    for p in range(Histogram_Corr_Image_Stack.shape[0]):
        Current_XY_Image = Histogram_Corr_Image_Stack[p]
        Cmin = np.min(Current_XY_Image)
        Cmax = np.max(Current_XY_Image)
        human_result[p]=(Current_XY_Image - Cmin)/(Cmax - Cmin)

In [ ]:
if print_process :
    plt.imshow(human_result[100], cmap='gray')

In [ ]:
if human_see :
    write_output(human_result, "human_see")